In [1]:
from dotenv import load_dotenv
load_dotenv()

False

# Tool Calling and Structured Output

For many applications, such as chatbots, models need to respond to users directly in natural language. However, there are scenarios where we need models to output in a structured format. 
For example, we might want to store the model output in a database and ensure that the output conforms to the database schema.

This need motivates the concept of structured output, where models can be instructed to respond with a particular output structure.

![Structured output](assets/structured_output-2c42953cee807dedd6e96f3e1db17f69.png "Structured output")

`Pydantic` is particularly useful for defining structured output schemas because it offers type hints and validation. Here's an example of a Pydantic schema:

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")
    

model = ChatOpenAI(model="gpt-4o", temperature=0)
# Bind responseformatter schema as a tool to the model
model_with_tools = model.bind_tools([ResponseFormatter])
# Invoke the model
ai_msg = model_with_tools.invoke("What is the powerhouse of the cell?")

ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HA6oEQS3vZLZPNmQa2e1tBEJ', 'function': {'arguments': '{"answer":"The powerhouse of the cell is the mitochondrion. Mitochondria are organelles that generate most of the cell\'s supply of adenosine triphosphate (ATP), which is used as a source of chemical energy.","followup_question":"What is the function of ATP in the cell?"}', 'name': 'ResponseFormatter'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 80, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6ec83003ad', 'id': 'chatcmpl-BElWB4kn7fOCIwo5FT9zkDv55ENIg', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cda3cc86-a462-42ba-8c91-0037036983a6-0', too

### Challenges with Structured Output

There are a few challenges when producing structured output with the above methods:

1. **Tool Argument Parsing**  
   When tool calling is used, tool call arguments need to be parsed from a dictionary back to the original schema.

2. **Enforcing Tool Use**  
   The model needs to be instructed to always use the tool when we want to enforce structured output, which is a provider-specific setting.

3. **JSON Mode Parsing**  
   When JSON mode is used, the output needs to be parsed into a JSON object.

With these challenges in mind, LangChain provides a helper function — `with_structured_output()` — to streamline the process.

![Structured output](assets/with_structured_output-4fd0fdc94f644554d52c6a8dee96ea21.png "Structured output")

In [35]:
# Bind the schema to the model
model_with_structure = model.with_structured_output(ResponseFormatter)
# Invoke the model
structured_output = model_with_structure.invoke("What is the powerhouse of the cell?")
# Get back the pydantic object
print(structured_output)
print(type(structured_output))

answer='The powerhouse of the cell is the **mitochondrion** (plural: mitochondria). Mitochondria are organelles found in the cells of most eukaryotic organisms. They are responsible for producing energy in the form of adenosine triphosphate (ATP) through a process known as cellular respiration. This energy is essential for various cellular functions and activities, making mitochondria crucial for cell survival and function.' followup_question='How do mitochondria produce energy?'
<class '__main__.ResponseFormatter'>


## Tools

Many AI applications interact directly with humans. In these cases, it is appropriate for models to respond in natural language. 

But what about cases where we want a model to also interact directly with systems, such as databases or an API? 
These systems often have a particular input schema; for example, APIs frequently have a required payload structure.
This need motivates the concept of tool calling. 

You can use tool calling to request model responses that match a particular schema.

![Tool Calling](assets/tool_call_example-2348b869f9a5d0d2a45dfbe614c177a4.png "Tool Calling")

### Key Concepts

1. **Tool Creation**  
   Use the `@tool` decorator to create a tool. A tool is an association between a function and its schema.

2. **Tool Binding**  
   The tool needs to be connected to a model that supports tool calling. This gives the model awareness of the tool and the associated input schema required by the tool.

3. **Tool Calling**  
   When appropriate, the model can decide to call a tool and ensure its response conforms to the tool's input schema.

4. **Tool Execution**  
   The tool can be executed using the arguments provided by the model.

### Tool creation

Creating a tool is as easy as using the `@tool` decorator on any function and add a description so the model can understand what the tool can do.

In [37]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

### Tool binding

As a specific example, let's take a function `multiply` and bind it as a tool to a model that supports tool calling.

In [39]:
model_with_tools = model.bind_tools([multiply])

### Tool Calling

A key principle of tool calling is that the model decides when to use a tool based on the input's relevance. The model doesn't always need to call a tool.

In [41]:
result = model_with_tools.invoke("Hello world!")
result

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 61, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6ec83003ad', 'id': 'chatcmpl-BElmJupDIEi3m8Akhbv0B1N8mWNtg', 'finish_reason': 'stop', 'logprobs': None}, id='run-f606775b-6917-4652-bbed-f9a615538247-0', usage_metadata={'input_tokens': 61, 'output_tokens': 11, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

However, if we pass an input relevant to the tool, the model should choose to call it:

In [43]:
result = model_with_tools.invoke("What is 2 multiplied by 3?")
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_XPikbEUUJN07AJgQN4QL0S6M',
  'type': 'tool_call'}]

### Tool execution
Tools implement the `Runnable` interface, which means that they can be invoked (e.g., `tool.invoke(args)`) directly.

In [45]:
multiply.invoke(result.tool_calls[0]['args'])

6

### Best Practices

When designing tools to be used by a model, keep the following in mind:

- **Use Models with Tool-Calling APIs**  
  Models that have explicit tool-calling APIs will be better at tool calling than non-fine-tuned models.

- **Name and Describe Tools Clearly**  
  Models perform better when tools have well-chosen, descriptive names and explanations.

- **Keep Tools Simple and Focused**  
  Simple, narrowly scoped tools are easier for models to use effectively than complex or multifunctional tools.

- **Limit the Number of Tools**  
  Asking the model to select from a large list of tools can be challenging and may reduce accuracy.
